In [ ]:
# prerequisites
import torch
import torchvision
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms
from torch.autograd import Variable
from torch.utils.data import Dataset, DataLoader
from torchvision.utils import save_image

from tqdm import tqdm_notebook as tqdm
from tqdm import tqdm as tqdmx

bs = 100
#Tools Dataset
dataset = torchvision.datasets.ImageFolder('../../data/',
                                           transform=transforms.Compose([
                                               transforms.Resize(28),
                                               transforms.CenterCrop(28),
                                               transforms.ToTensor()
                                           ]))

data_loader = DataLoader(dataset=dataset,
                         batch_size=bs, 
                         shuffle=False)

In [ ]:
from notify_run import Notify
notify = Notify()
notify.register()

In [ ]:
# Device configuration
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device

In [ ]:
class VAE(nn.Module):
    def __init__(self, x_dim, h_dim1, h_dim2, z_dim):
        super(VAE, self).__init__()
        
        # encoder part
        self.fc1 = nn.Linear(x_dim, h_dim1)
        self.fc2 = nn.Linear(h_dim1, h_dim2)
        self.fc31 = nn.Linear(h_dim2, z_dim)
        self.fc32 = nn.Linear(h_dim2, z_dim)
        # decoder part
        self.fc4 = nn.Linear(z_dim, h_dim2)
        self.fc5 = nn.Linear(h_dim2, h_dim1)
        self.fc6 = nn.Linear(h_dim1, x_dim)
        
    def encoder(self, x):
        h = F.relu(self.fc1(x))
        h = F.relu(self.fc2(h))
        return self.fc31(h), self.fc32(h) # mu, log_var
    
    def sampling(self, mu, log_var):
        std = torch.exp(0.5*log_var)
        eps = torch.randn_like(std)
        return eps.mul(std).add_(mu) # return z sample
        
    def decoder(self, z):
        h = F.relu(self.fc4(z))
        h = F.relu(self.fc5(h))
        return torch.sigmoid(self.fc6(h)) 
    
    def forward(self, x):
        mu, log_var = self.encoder(x.view(-1, 784))
        z = self.sampling(mu, log_var)
        return self.decoder(z), mu, log_var

# build model
z_dim = 2
vae = VAE(x_dim=784, h_dim1= 512, h_dim2=256, z_dim=z_dim)

In [ ]:
if torch.cuda.is_available():
    vae.cuda()

In [ ]:
optimizer = optim.Adam(vae.parameters())
# return reconstruction error + KL divergence losses
def loss_function(recon_x, x, mu, log_var):
    BCE = F.binary_cross_entropy(recon_x, x.view(-1, 784), reduction='sum')
    KLD = -0.5 * torch.sum(1 + log_var - mu.pow(2) - log_var.exp())
    return BCE + KLD

In [ ]:
for epoch in tqdm(range(1, 51)):
    notify.send('Training epoch {}'.format(epoch))
    vae.train()
    train_loss = 0
    for batch_idx, (data, _) in enumerate(tqdm(data_loader)):
        if torch.cuda.is_available():
            data = data.cuda()
        optimizer.zero_grad()

        recon_batch, mu, log_var = vae(data)
        loss = loss_function(recon_batch, data, mu, log_var)

        loss.backward()
        train_loss += loss.item()
        optimizer.step()

    tqdmx.write('Epoch: {} Average loss: {:.4f}'.format(epoch, train_loss / len(data_loader.dataset)))

In [ ]:
torch.save(vae.state_dict(), "./mnist-vae-tools_zdim2.torch")
notify.send("Saved model")

In [ ]:
vae.load_state_dict(torch.load('./mnist-vae-tools_zdim2.torch'))

In [ ]:
import numpy as np
# Plot the image here using matplotlib.
def plot_image(tensor, save=None):
    plt.figure(figsize=(20, 5))
    # imshow needs a numpy array with the channel dimension
    # as the the last dimension so we have to transpose things.
    plt.imshow(tensor.numpy().transpose(1, 2, 0))
    
    if save is not None:
        plt.savefig(save)
    
    plt.show()


In [ ]:
summary = []
for i in range(0, 1000, 50):
    fixed_x = dataset[i][0]
    recon_x = vae(fixed_x.cuda())[0].view(3, 28, 28)
    compare = torch.cat([fixed_x.cuda(), recon_x], dim=1)
    summary.append(compare)
    
plot_image(torch.cat(summary, dim=2).cpu().detach(), save='summary_zdim2.png')

In [ ]:
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
from scipy.stats import norm

# display a 2D manifold of the digits
n = 15  # figure with 8x8 digits
digit_size = 28

# linearly spaced coordinates on the unit square were transformed
# through the inverse CDF (ppf) of the Gaussian to produce values
# of the latent variables z, since the prior of the latent space
# is Gaussian
u_grid = np.dstack(np.meshgrid(np.linspace(0.05, 0.95, n),
                               np.linspace(0.05, 0.95, n)))
z_grid = norm.ppf(u_grid)

x_decoded = vae.decoder(torch.from_numpy(z_grid.reshape(n*n, 2)).float().cuda())
x_decoded = x_decoded.reshape(n, n, digit_size, digit_size)

plt.figure(figsize=(10, 10))
plt.imshow(np.block(list(map(list, x_decoded.detach().cpu().numpy()))), cmap='gray')
plt.show()